# Tata Consultancy Services - Historical Data - Scraper
## Author : George M Cherian

## Scraping Script

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# Data to Post (PayLoad)
data = { 
"curr_id": "18420",
"smlID": "1166518",
"header": "TCS Historical Data",
"st_date": "04/01/2016",
"end_date": "11/19/2020", # Set Today's Date (MM/DD/YYYY)
"interval_sec": "Daily",
"sort_col": "date",
"sort_ord": "DESC",
"action": "historical_data"
}

In [3]:
# Adding a User Agent and specifying AJAX request
urlheader = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}


In [4]:
with requests.Session() as sess:
    req = sess.post("https://www.investing.com/instruments/HistoricalDataAjax", data=data, headers=urlheader)
    soup = BeautifulSoup(req.content, "lxml")

In [5]:
# Finding the desired table
table = soup.find("table",id="curr_table")

In [6]:
headings = []
rows = []

In [7]:
# Extracting Table Headings
for row in table.select('th'):
    headings.append(row.getText().replace('\n',' '))
    

In [8]:
headings


['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']

In [9]:
#Extracting row values
for row in table.select('tr'):
    temp = []
    for column in row.select('td'):
        temp.append(column.getText())
    rows.append(temp)
        
    

In [10]:
rows = rows[1:]

## Data Processing

In [11]:
import pandas as pd

In [12]:
# Converting lists to pandas DataFrame
df = pd.DataFrame(columns=headings,data=rows)

In [13]:
# Dropping all columns that are not needed
df.drop(['Open','High','Low','Vol.','Change %'],axis=1,inplace=True)

In [14]:
# Converting Date in string format to datetime object for querying
df['Date'] = df['Date'].apply(pd.to_datetime)
int_prices = []

In [15]:
for val in df['Price']:
    int_prices.append(val.replace(',',''))
    
    

In [16]:
df['Price'] = int_prices

In [17]:
#Converting price in string format to float for querying
df['Price'] = df['Price'].apply(pd.to_numeric)

In [18]:
#Updated DataFrame
df

,Date,Price
0,2020-11-19,2655.45
1,2020-11-18,2626.65
2,2020-11-17,2666.05
3,2020-11-13,2656.85
4,2020-11-12,2671.05
...,...,...
1143,2016-04-07,1211.25
1144,2016-04-06,1216.69
1145,2016-04-05,1207.31
1146,2016-04-04,1211.47


## Dumping DataFrame into MySQL

In [19]:
import pymysql
import sqlalchemy

In [20]:
sqlEngine = sqlalchemy.create_engine('mysql+pymysql://root:@127.0.0.1/student', pool_recycle=3600)

In [21]:
db = sqlEngine.connect()

In [22]:
db

In [23]:
tableName = "TCS"

In [24]:
frame = df.to_sql(tableName, db, if_exists='fail');

In [25]:
output = pd.read_sql("SELECT * FROM student.TCS",db);

In [26]:
output

,index,Date,Price
0,0,2020-11-19,2655.45
1,1,2020-11-18,2626.65
2,2,2020-11-17,2666.05
3,3,2020-11-13,2656.85
4,4,2020-11-12,2671.05
...,...,...,...
1143,1143,2016-04-07,1211.25
1144,1144,2016-04-06,1216.69
1145,1145,2016-04-05,1207.31
1146,1146,2016-04-04,1211.47


In [27]:
db.close()